In [27]:
import pandas as pd 
import numpy as np 
import math
import datetime
from nsepython import *
import matplotlib.pyplot as plt
from scipy.stats import norm 


In [35]:

def BlackScholesPricingModel(Spot_Price, Strike_Price, expiry_date, risk_free_rate, volatility):
    S = Spot_Price
    K = Strike_Price
    today = datetime.datetime.now()
    day_diff = (expiry_date - today).days
    if day_diff <= 0:
        raise ValueError("Expiry date must be in the future")
    T = day_diff / 365
    r = risk_free_rate
    sigma = volatility/100

    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    Option_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return Option_price

In [36]:
import numpy as np
from datetime import datetime
from nsepython import *

# --- Fetch option chain ---
ticker = 'TCS'
option_chain = nse_optionchain_scrapper(ticker)
records = option_chain['records']['data']

# --- Get current stock price ---
S0 = option_chain['records']['underlyingValue']

# --- Filter only calls ---
call_options = [item for item in records if 'CE' in item]

# --- Pick ATM option ---
atm_option = min(call_options, key=lambda x: abs(x['CE']['strikePrice'] - S0))
K = atm_option['CE']['strikePrice']
market_price = atm_option['CE']['lastPrice']
implied_volatility = atm_option['CE']['impliedVolatility']  # in % already
expiry = atm_option['CE']['expiryDate']

# --- Convert expiry to datetime object ---
expiry_date = datetime.datetime.strptime(expiry, '%d-%b-%Y')

# --- Compute Black-Scholes price ---
bsp_price = BlackScholesPricingModel(Spot_Price=S0, Strike_Price=K,
                                     expiry_date=expiry_date,
                                     risk_free_rate=0.0625,
                                     volatility=implied_volatility)

# --- Print results ---
print(f"Stock Price: ₹{S0}")
print(f"Strike Price: ₹{K}")
print(f"Expiry Date: {expiry}")
print(f"Market Price: ₹{market_price}")
print(f"Black Scholes Model Price: ₹{bsp_price:.2f}")


Stock Price: ₹2903
Strike Price: ₹2900
Expiry Date: 28-Oct-2025
Market Price: ₹74.85
Black Scholes Model Price: ₹69.34
